<a href="https://colab.research.google.com/github/georgeaidinis/NTUA/blob/master/NN/Lab%201/Copy_of_Lab_1_Big_NN_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Αϊδίνης Γιώργος 03116031

Κολιός Παναγιώτης 03116100

---

Ομάδα M.B.8

---


#Lab 1: Επιβλεπόμενη Μάθηση - Ταξινόμηση - Μεγάλο Dataset (B10 - Epileptic Seizure Recognition)

In [41]:
!pip install --upgrade pip #upgrade pip package installer
!pip install scikit-learn --upgrade #upgrade scikit-learn package
!pip install numpy --upgrade #upgrade numpy package
!pip install pandas --upgrade #--upgrade #upgrade pandas package

Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.3.1)
Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.6/dist-packages (0.22)
Requirement already up-to-date: numpy in /usr/local/lib/python3.6/dist-packages (1.17.4)
Requirement already up-to-date: pandas in /usr/local/lib/python3.6/dist-packages (0.25.3)


Multiclass Classification: Το συγκεκριμένο dataset χρησιμοποιείται και για multiclass και για binary (1 vs 2-5) classification. Λόγω του παραπάνω, και καθώς στο small dataset κάναμε binary classification επιλέξαμε να κάνουμε multiclass σε αυτό.

1. Έγινε καταγραφή της δραστηριότητας του εγκεφάλου 500 ατόμων για 23.6 δευτερόλεπτα/καταγραφή. Στόχος είναι η λήψη σωστής απόφασης περί του αν το άτομο βρίσκεται σε επιληπτική κρίση ή όχι. Έγινε δειγματοληψία κάθε καταγραφής, η οποία οδήγησε σε 4097 δείγματα. Τα 4097 δείγματα χωρίστηκαν σε 23 κομμάτια, με το καθένα να περιέχει 178 σημεία, που αντιστοιχούν σε 1 δευτερόλεπτο καταγραφής. Έτσι δημιουργήθηκαν 23*500 = 11500 γραμμές-δείγματα, καθένα από τα οποία αποτελείται από 178 σημεία που αντιστοιχούν σε 1 δευτερόλεπτο και αποτελόυν τη διάσταση των δεδομένων εισόδου. Η τελευταία στήλη περιέχει τις ετικέτες, οι οποίες παίρνουν τιμές 1-5, με την περίπτωση 1 να αφορά τις περιπτώσεις όπου το άτομο είχε επιληπτική κρίση και τις 2-5 τις περιπτώσεις όπου δεν είχε. 


In [0]:
from urllib.request import urlretrieve
import os
from zipfile import ZipFile

def download(url, file):
    if not os.path.isfile(file):
        print("Download file... " + file + " ...")
        urlretrieve(url,file)
        print("File downloaded")

In [43]:
download("http://archive.ics.uci.edu/ml/machine-learning-databases/00388/data.csv","ESR.csv")
print("All the files are downloaded")

All the files are downloaded


In [44]:
!ls

ESR.csv  sample_data


In [0]:
import pandas as pd
import numpy as np

df = pd.read_csv("ESR.csv")

2. Όπως είπαμε παραπάνω και όπως φαίνεται παρακάτω, υπάρχουν 11500 δείγματα, το καθένα με 178 χαρακτηριστικά, τα οποία αφορούν τις μεταβολές της ηλεκτρικής τάσης των σημάτων στους νευρώνες του ανθρωπίνου εγκεφάλου. Συνεπώς είναι διατεταγμένα. Επίσης είναι ακέραιοι αριθμοί.
3. Υπάρχουν επικεφαλίδες στην πρώτη γραμμή και στοιχεία για το εκάστοτε δείγμα στην πρώτη στήλη, τα οποία θα πρέπει να αφαιρεθούν.
4. 

In [46]:
print(df.shape)

(11500, 180)


In [0]:
df #print the dataset

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,...,X140,X141,X142,X143,X144,X145,X146,X147,X148,X149,X150,X151,X152,X153,X154,X155,X156,X157,X158,X159,X160,X161,X162,X163,X164,X165,X166,X167,X168,X169,X170,X171,X172,X173,X174,X175,X176,X177,X178,y
0,X21.V1.791,135,190,229,223,192,125,55,-9,-33,-38,-10,35,64,113,152,164,127,50,-47,-121,-138,-125,-101,-50,11,39,24,48,64,46,13,-19,-61,-96,-130,-132,-116,-115,-71,...,54,90,111,107,64,32,18,-25,-69,-65,-44,-33,-57,-88,-114,-130,-114,-83,-53,-79,-72,-85,-109,-98,-72,-65,-63,-11,10,8,-17,-15,-31,-77,-103,-127,-116,-83,-51,4
1,X15.V1.924,386,382,356,331,320,315,307,272,244,232,237,258,212,2,-267,-605,-850,-1001,-1109,-1090,-967,-746,-464,-152,118,318,427,473,485,447,397,339,312,314,326,335,332,324,310,...,27,146,229,269,297,307,303,305,306,307,280,231,159,85,51,43,62,63,63,69,89,123,136,127,102,95,105,131,163,168,164,150,146,152,157,156,154,143,129,1
2,X8.V1.1,-32,-39,-47,-37,-32,-36,-57,-73,-85,-94,-99,-94,-96,-104,-103,-92,-75,-69,-69,-53,-37,-14,-10,-39,-78,-102,-98,-80,-54,-40,-35,-39,-32,-13,7,34,41,33,6,...,-82,-107,-126,-124,-108,-84,-68,-61,-56,-63,-62,-33,1,28,45,37,48,62,80,66,23,-11,-39,-44,-42,-45,-48,-42,-6,29,57,64,48,19,-12,-30,-35,-35,-36,5
3,X16.V1.60,-105,-101,-96,-92,-89,-95,-102,-100,-87,-79,-72,-68,-74,-80,-83,-73,-68,-61,-58,-59,-64,-79,-84,-97,-94,-84,-77,-75,-72,-68,-76,-76,-72,-67,-69,-69,-69,-67,-68,...,-69,-66,-74,-69,-61,-51,-45,-45,-49,-58,-64,-78,-80,-90,-87,-83,-78,-64,-38,-22,-29,-42,-51,-68,-71,-69,-69,-74,-74,-80,-82,-81,-80,-77,-85,-77,-72,-69,-65,5
4,X20.V1.54,-9,-65,-98,-102,-78,-48,-16,0,-21,-59,-90,-103,-84,-43,-9,3,-21,-60,-96,-103,-75,-29,14,55,78,73,28,-13,-43,-68,-78,-75,-55,-41,-19,-20,-29,-36,-20,...,38,26,10,-4,-13,-8,0,10,19,29,57,63,45,7,-13,-23,-9,9,11,3,-1,-2,4,18,27,27,14,15,11,10,4,2,-12,-32,-41,-65,-83,-89,-73,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11495,X22.V1.114,-22,-22,-23,-26,-36,-42,-45,-42,-45,-49,-57,-64,-73,-79,-76,-70,-63,-57,-57,-50,-45,-34,-33,-32,-30,-24,-24,-18,-9,-2,0,2,-3,-5,-13,-23,-39,-53,-59,...,45,55,60,69,68,58,54,44,40,38,39,39,32,23,22,14,13,15,18,20,20,25,28,29,29,29,26,24,24,20,15,16,12,5,-1,-18,-37,-47,-48,2
11496,X19.V1.354,-47,-11,28,77,141,211,246,240,193,136,78,8,-66,-132,-180,-210,-227,-225,-212,-192,-168,-144,-117,-88,-54,-21,6,29,46,60,76,92,110,128,152,171,150,91,21,...,69,58,38,16,25,75,165,260,343,406,456,471,461,412,319,175,-5,-171,-293,-357,-378,-370,-346,-316,-278,-241,-201,-162,-126,-94,-65,-33,-7,14,27,48,77,117,170,1
11497,X8.V1.28,14,6,-13,-16,10,26,27,-9,4,14,-1,-10,14,44,77,61,42,32,29,22,26,35,81,66,50,57,91,121,111,73,38,23,35,21,11,-12,2,17,61,...,-90,-62,-38,-40,-21,-23,-11,-4,-9,-22,-42,-49,-48,-40,-40,-46,-43,-67,-67,-50,-18,-14,-39,-74,-86,-75,-68,-57,-78,-42,-65,-48,-61,-62,-67,-30,-2,-1,-8,5
11498,X10.V1.932,-40,-25,-9,-12,-2,12,7,19,22,29,22,6,1,-28,-37,-35,-35,-45,-64,-105,-140,-157,-157,-147,-153,-147,-126,-112,-83,-56,-41,-40,-38,-34,-47,-41,-40,-42,-49,...,-98,-77,-60,-73,-88,-97,-118,-108,-100,-97,-91,-109,-122,-134,-137,-107,-95,-67,-54,-40,-31,-19,7,22,46,68,87,97,105,114,121,135,148,143,116,86,68,59,55,3


In [0]:
print ("We have ",len(df.columns), " attributes.")
for i in range(0, len(df.columns)):
    print('{:<10}{:<40}{:<10}{:<20}'.format(str(i+1), str(df.columns[i]),"type: ", str(df.dtypes[df.columns[i]])))

5. Διαγράφουμε την πρώτη κολώνα, ώστε όλες οι κολώνες εκτός της τελευταίας να περιέχουν τιμές των χαρακτηριστικών, με την τελευταία να περιέχει τις ετικέτες.

In [48]:
df = df.drop(df.columns[[0]], axis=1)
df

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,...,X140,X141,X142,X143,X144,X145,X146,X147,X148,X149,X150,X151,X152,X153,X154,X155,X156,X157,X158,X159,X160,X161,X162,X163,X164,X165,X166,X167,X168,X169,X170,X171,X172,X173,X174,X175,X176,X177,X178,y
0,135,190,229,223,192,125,55,-9,-33,-38,-10,35,64,113,152,164,127,50,-47,-121,-138,-125,-101,-50,11,39,24,48,64,46,13,-19,-61,-96,-130,-132,-116,-115,-71,-14,...,54,90,111,107,64,32,18,-25,-69,-65,-44,-33,-57,-88,-114,-130,-114,-83,-53,-79,-72,-85,-109,-98,-72,-65,-63,-11,10,8,-17,-15,-31,-77,-103,-127,-116,-83,-51,4
1,386,382,356,331,320,315,307,272,244,232,237,258,212,2,-267,-605,-850,-1001,-1109,-1090,-967,-746,-464,-152,118,318,427,473,485,447,397,339,312,314,326,335,332,324,310,312,...,27,146,229,269,297,307,303,305,306,307,280,231,159,85,51,43,62,63,63,69,89,123,136,127,102,95,105,131,163,168,164,150,146,152,157,156,154,143,129,1
2,-32,-39,-47,-37,-32,-36,-57,-73,-85,-94,-99,-94,-96,-104,-103,-92,-75,-69,-69,-53,-37,-14,-10,-39,-78,-102,-98,-80,-54,-40,-35,-39,-32,-13,7,34,41,33,6,-15,...,-82,-107,-126,-124,-108,-84,-68,-61,-56,-63,-62,-33,1,28,45,37,48,62,80,66,23,-11,-39,-44,-42,-45,-48,-42,-6,29,57,64,48,19,-12,-30,-35,-35,-36,5
3,-105,-101,-96,-92,-89,-95,-102,-100,-87,-79,-72,-68,-74,-80,-83,-73,-68,-61,-58,-59,-64,-79,-84,-97,-94,-84,-77,-75,-72,-68,-76,-76,-72,-67,-69,-69,-69,-67,-68,-69,...,-69,-66,-74,-69,-61,-51,-45,-45,-49,-58,-64,-78,-80,-90,-87,-83,-78,-64,-38,-22,-29,-42,-51,-68,-71,-69,-69,-74,-74,-80,-82,-81,-80,-77,-85,-77,-72,-69,-65,5
4,-9,-65,-98,-102,-78,-48,-16,0,-21,-59,-90,-103,-84,-43,-9,3,-21,-60,-96,-103,-75,-29,14,55,78,73,28,-13,-43,-68,-78,-75,-55,-41,-19,-20,-29,-36,-20,1,...,38,26,10,-4,-13,-8,0,10,19,29,57,63,45,7,-13,-23,-9,9,11,3,-1,-2,4,18,27,27,14,15,11,10,4,2,-12,-32,-41,-65,-83,-89,-73,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11495,-22,-22,-23,-26,-36,-42,-45,-42,-45,-49,-57,-64,-73,-79,-76,-70,-63,-57,-57,-50,-45,-34,-33,-32,-30,-24,-24,-18,-9,-2,0,2,-3,-5,-13,-23,-39,-53,-59,-63,...,45,55,60,69,68,58,54,44,40,38,39,39,32,23,22,14,13,15,18,20,20,25,28,29,29,29,26,24,24,20,15,16,12,5,-1,-18,-37,-47,-48,2
11496,-47,-11,28,77,141,211,246,240,193,136,78,8,-66,-132,-180,-210,-227,-225,-212,-192,-168,-144,-117,-88,-54,-21,6,29,46,60,76,92,110,128,152,171,150,91,21,-29,...,69,58,38,16,25,75,165,260,343,406,456,471,461,412,319,175,-5,-171,-293,-357,-378,-370,-346,-316,-278,-241,-201,-162,-126,-94,-65,-33,-7,14,27,48,77,117,170,1
11497,14,6,-13,-16,10,26,27,-9,4,14,-1,-10,14,44,77,61,42,32,29,22,26,35,81,66,50,57,91,121,111,73,38,23,35,21,11,-12,2,17,61,88,...,-90,-62,-38,-40,-21,-23,-11,-4,-9,-22,-42,-49,-48,-40,-40,-46,-43,-67,-67,-50,-18,-14,-39,-74,-86,-75,-68,-57,-78,-42,-65,-48,-61,-62,-67,-30,-2,-1,-8,5
11498,-40,-25,-9,-12,-2,12,7,19,22,29,22,6,1,-28,-37,-35,-35,-45,-64,-105,-140,-157,-157,-147,-153,-147,-126,-112,-83,-56,-41,-40,-38,-34,-47,-41,-40,-42,-49,-56,...,-98,-77,-60,-73,-88,-97,-118,-108,-100,-97,-91,-109,-122,-134,-137,-107,-95,-67,-54,-40,-31,-19,7,22,46,68,87,97,105,114,121,135,148,143,116,86,68,59,55,3


6. Δεν υπάρχουν απουσιάζουσες τιμές.
7. Χρησιμοποιούμε την bincount για να μετρήσουμε τη συχνότητα των κατηγοριών. Παρατηρούμε οτι έχουμε ενα εξαιρετικά ισορροπημένο dataset. Και οι 5 κατηγορίες είναι ισοπληθείς. 


In [50]:
labels = df.iloc[:, 178]
df = df.iloc[:, 0:178]   #remove lables from set
print("frequencies:", np.bincount(labels)[1:])

frequencies: [2300 2300 2300 2300 2300]


8. Διαχωρίζουμε σε train και test set. Οι τιμές των χαρακτηριστικών αφορούν τα εγκεφαλικά σήματα (τάσεις), άρα είναι διατεταγμένα.

In [53]:
from sklearn.model_selection import train_test_split

train, test, train_labels, test_labels = train_test_split(df, labels, test_size=0.3)
print(len(train))

8050


# Γ. Baseline Classification

Εκπαιδεύουμε τους classifiers με τις default τιμές για να δούμε συγκρίνουμε τα αποτέλεσματα πριν και μετά την προεργασία. <br>

Αρχίζουμε με τους Dummy Classifiers. Παρατηρούμε οτι επιτυγχάνουμε σε όλους περίπου 20% επιτυχία, όπως ήταν αναμενόμενο καθώς έχουμε 5 κατηγορίες.

In [76]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

dc_uniform = DummyClassifier(strategy="uniform")
dc_constant_1 = DummyClassifier(strategy="constant", constant=1)
dc_constant_2 = DummyClassifier(strategy="constant", constant=2)
dc_most_frequent = DummyClassifier(strategy="most_frequent")
dc_stratified = DummyClassifier(strategy="stratified")

#using fit to train the classifiers
model_uniform = dc_uniform.fit(train, train_labels)
model_constant_1 = dc_constant_1.fit(train, train_labels)
model_constant_2 = dc_constant_2.fit(train, train_labels)
model_most_frequent = dc_most_frequent.fit(train, train_labels)
model_stratified = dc_stratified.fit(train, train_labels)

#now we make our predictions
preds_uniform = dc_uniform.predict(test)
preds_constant_1 = dc_constant_1.predict(test)
preds_constant_2 = dc_constant_2.predict(test)
preds_most_frequent = dc_most_frequent.predict(test)
preds_stratified = dc_stratified.predict(test)

#print prediction accuracy
print("Uniform Classifier: ", accuracy_score(test_labels, preds_uniform))
print("Constant Classifier (1): ", accuracy_score(test_labels, preds_constant_1))
print("Constant Classifier (2): ", accuracy_score(test_labels, preds_constant_2))
print("Most Frequent Classifier: ", accuracy_score(test_labels, preds_most_frequent))
print("Stratified Classifier: ", accuracy_score(test_labels, preds_stratified))

Uniform Classifier:  0.20347826086956522
Constant Classifier (1):  0.19768115942028985
Constant Classifier (2):  0.20347826086956522
Most Frequent Classifier:  0.19710144927536233
Stratified Classifier:  0.1855072463768116


Εκτυπώνουμε τον confusion matrix. Ο $C$ είναι ο πίνακας για τον οποίο ισχύει οτι $C_{i, j}$ είναι τα δείγματα της κατηγορίας $i$ που ταξινομήθηκαν στην $j$. Όπως φαίνεται οι Uniform και Stratified προβλέπουν οποιαδήποτε κατηγορία, ενώ οι άλλοι 3 είτε επιλέγουν σταθερά μία κατηγορία είτε αυτή με τα περισσότερα δείγματα. Μάλιστα, από τη στιγμή που τα δείγματα στο αρχικό data set είναι ίσα μεταξύ τους, τα πολυπληθέστερα δείγματα στο train set θα είναι τα λιγότερο πολυπληθή στο test set. Άρα ο classifier αυτός θα έχει χειρότερη επίδοση από τους constant classifiers, όπως φαίνεται και από τον confusion matrix.

In [0]:
from sklearn.metrics import confusion_matrix

#produce confusion matrices
cnf_matrix_uniform = confusion_matrix(test_labels, preds_uniform)
cnf_matrix_constant_1 = confusion_matrix(test_labels, preds_constant_1)
cnf_matrix_constant_2 = confusion_matrix(test_labels, preds_constant_2)
cnf_matrix_most_frequent = confusion_matrix(test_labels, preds_most_frequent)
cnf_matrix_stratified = confusion_matrix(test_labels, preds_stratified)

#print confusion matrices
print("Uniform Classifier Confusion Matrix\n")
print(cnf_matrix_uniform, "\n")
print("Constant Classifier (1) Confusion Matrix\n")
print(cnf_matrix_constant_1, "\n")
print("Constant Classifier (2) Confusion Matrix\n")
print(cnf_matrix_constant_2, "\n")
print("Most Frequent Classifier Confusion Matrix\n")
print(cnf_matrix_most_frequent, "\n")
print("Stratified Classifier Confusion Matrix\n")
print(cnf_matrix_stratified, "\n")


Εκτύπωση f1-micro average και f1-macro average.

In [94]:
from sklearn.metrics import precision_recall_fscore_support
import warnings
warnings.filterwarnings("ignore")

#f1-micro
print('\033[1m' + "f1-micro" + '\033[0m')
print("Uniform Classifier: ", precision_recall_fscore_support(test_labels, preds_uniform, average='micro'))
print("Constant Classifier (1): ", precision_recall_fscore_support(test_labels, preds_constant_1, average='micro'))
print("Constant Classifier (2): ", precision_recall_fscore_support(test_labels, preds_constant_2, average='micro'))
print("Most Frequent Classifier: ", precision_recall_fscore_support(test_labels, preds_most_frequent, average='micro'))
print("Stratified Classifier: ", precision_recall_fscore_support(test_labels, preds_stratified, average='micro'), "\n")


#f1-macro
print('\033[1m' + "f1-macro" + '\033[0m')
print("Uniform Classifier: ", precision_recall_fscore_support(test_labels, preds_uniform, average='macro'))
print("Constant Classifier (1): ", precision_recall_fscore_support(test_labels, preds_constant_1, average='macro'))
print("Constant Classifier (2): ", precision_recall_fscore_support(test_labels, preds_constant_1, average='macro'))
print("Most Frequent Classifier: ", precision_recall_fscore_support(test_labels, preds_most_frequent, average='macro'))
print("Stratified Classifier: ", precision_recall_fscore_support(test_labels, preds_stratified, average='macro'))



f1-micro
Uniform Classifier:  (0.20347826086956522, 0.20347826086956522, 0.20347826086956522, None)
Constant Classifier (1):  (0.19768115942028985, 0.19768115942028985, 0.19768115942028985, None)
Constant Classifier (2):  (0.20347826086956522, 0.20347826086956522, 0.20347826086956522, None)
Most Frequent Classifier:  (0.19710144927536233, 0.19710144927536233, 0.1971014492753623, None)
Stratified Classifier:  (0.1855072463768116, 0.1855072463768116, 0.1855072463768116, None) 

f1-macro
Uniform Classifier:  (0.20355172484328876, 0.20347134909521153, 0.20347818207311566, None)
Constant Classifier (1):  (0.03953623188405797, 0.2, 0.0660212971926428, None)
Constant Classifier (2):  (0.03953623188405797, 0.2, 0.0660212971926428, None)
Most Frequent Classifier:  (0.03942028985507247, 0.2, 0.06585956416464891, None)
Stratified Classifier:  (0.185638230006699, 0.1854588107981207, 0.18547416852196202, None)
